# Additional training functions

`train` provides a number of extension methods that are added to `Learner` (see below for a list and details), along with three simple callbacks:

- `ShowGraph`
- `GradientClipping`
- `BnFreeze`

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.train import *
from fastai.docs import *
from fastai import *

## `Learner` extension methods

These methods are automatically added to all `Learner` objects created after importing this module. They provide convenient access to a number of callbacks, without requiring them to be manually created.

In [ ]:
show_doc(fit_one_cycle)

#### <a id=fit_one_cycle></a>`fit_one_cycle`
> `fit_one_cycle`(`learn`:`Learner`, `cyc_len`:`int`, `max_lr`:`Union`\[`float`, `Collection`\[`float`\], `slice`\]=`slice(None, 0.003, None)`, `moms`:`Point`=`(0.95, 0.85)`, `div_factor`:`float`=`25.0`, `pct_start`:`float`=`0.3`, `wd`:`float`=`None`, `kwargs`)


Fit a model following the 1cycle policy. <a href="https://github.com/fastai/fastai/blob/master/fastai/train.py#L11">[source]</a>

Fit a model with 1cycle training. See `OneCycleScheduler` for details.

In [ ]:
show_doc(lr_find)

#### <a id=lr_find></a>`lr_find`
> `lr_find`(`learn`:`Learner`, `start_lr`:`float`=`1e-05`, `end_lr`:`float`=`10`, `num_it`:`int`=`100`, `kwargs`:`Any`)


Explore lr from `start_lr` to `end_lr` over `num_it` iterations in `learn`. <a href="https://github.com/fastai/fastai/blob/master/fastai/train.py#L20">[source]</a>

See `LRFinder` for details.

In [ ]:
show_doc(to_fp16)

#### <a id=to_fp16></a>`to_fp16`
> `to_fp16`(`learn`:`Learner`, `loss_scale`:`float`=`512.0`, `flat_master`:`bool`=`False`) -> `Learner`


Transform `learn` in FP16 precision. <a href="https://github.com/fastai/fastai/blob/master/fastai/train.py#L26">[source]</a>

See `MixedPrecision` for details.

In [ ]:
show_doc(mixup)

#### <a id=mixup></a>`mixup`
> `mixup`(`learn`:`Learner`, `alpha`:`float`=`0.4`, `stack_x`:`bool`=`False`, `stack_y`:`bool`=`True`) -> `Learner`


Add mixup https://arxiv.org/abs/1710.09412 to `learn`. <a href="https://github.com/fastai/fastai/blob/master/fastai/train.py#L33">[source]</a>

See `MixUpCallback` for more details.

A last extension method comes from the module tta.

In [ ]:
show_doc(Learner.TTA, full_name='TTA')

#### <a id=TTA></a>`TTA`
> `TTA`(`learn`:`Learner`, `beta`:`float`=`0.4`, `scale`:`float`=`1.35`, `is_test`:`bool`=`False`) -> `Tensors`
<a href="https://github.com/fastai/fastai/blob/master/fastai/tta.py#L45">[source]</a>

Applies Test Time Augmentation to `learn` on the validation set or the test test (depending on `is_test`). We take the average of our regular predictions (with a weight `beta`) with the average of predictions obtained thourh augmented versions of the training set (with a weight `1-beta`). The transforms decided for the training set are applied with a few changes `scale` controls the scale for zoom (which isn't random), the cropping isn't random but we make sure to get the four corners of the image. Flipping isn't random but applied once on each of those corner images (so that makes 8 augmented versions total).

In [ ]:
show_doc(ShowGraph)

## <a id=ShowGraph></a>`class` `ShowGraph`
> `ShowGraph`(`learn`:`Learner`) :: `LearnerCallback`


Update a graph of learner stats and metrics after each epoch. <a href="https://github.com/fastai/fastai/blob/master/fastai/train.py#L44">[source]</a>

In [ ]:
data = get_mnist()

In [ ]:
learn = ConvLearner(data, tvm.resnet18, metrics=accuracy, callback_fns=ShowGraph)
learn.fit(3)

![Training graph](imgs/train_graph.gif)

In [ ]:
show_doc(ShowGraph.on_epoch_end, doc_string=False)

#### <a id=on_epoch_end></a>`on_epoch_end`
> `on_epoch_end`(`n_epochs`:`int`, `last_metrics`:`MetricsList`, `kwargs`) -> `bool`
<a href="https://github.com/fastai/fastai/blob/master/fastai/train.py#L46">[source]</a>

If we have `last_metrics`, plot them in `self.pbar`. Set the size of the graph with `n_epochs`.

In [ ]:
show_doc(GradientClipping)

## <a id=GradientClipping></a>`class` `GradientClipping`
> `GradientClipping`(`learn`:`Learner`, `clip`:`float`) :: `LearnerCallback`


To do gradient clipping during training. <a href="https://github.com/fastai/fastai/blob/master/fastai/train.py#L64">[source]</a>

Clips gradient at a maximum absolute value of `clip` during training. For instance:

In [ ]:
learn = ConvLearner(data, tvm.resnet18, metrics=accuracy,
    callback_fns=partial(GradientClipping, clip=0.1))
learn.fit(1)

Total time: 00:05
epoch  train loss  valid loss  accuracy
0      0.080691    0.043826    0.986261  (00:05)



In [ ]:
show_doc(GradientClipping.on_backward_end, doc_string=False)

#### <a id=on_backward_end></a>`on_backward_end`
> `on_backward_end`(`kwargs`)
<a href="https://github.com/fastai/fastai/blob/master/fastai/train.py#L68">[source]</a>

Clip the gradients after they are computed but before the optimizer step.

In [ ]:
show_doc(BnFreeze)

## <a id=BnFreeze></a>class `BnFreeze`
`BnFreeze`(`learn`:`Learner`) :: `LearnerCallback`


Freezes moving average statistics in all batchnorm layers <a href="https://github.com/fastai/fastai/blob/master/fastai/train.py#L57">[source]</a>

For batchnorm layers where `requires_grad==False`, you generally don't want to update their moving average statistics, in order to avoid the model's statistics getting out of sync with its pre-trained weights. You can add this callback to automate this freezing of statistics (internally, it calls `eval` on these layers).

In [ ]:
learn = ConvLearner(data, tvm.resnet18, metrics=accuracy, callback_fns=BnFreeze)
learn.fit(1)

Total time: 00:05
epoch  train loss  valid loss  accuracy
0      0.085482    0.052628    0.984789  (00:05)



In [ ]:
show_doc(BnFreeze.on_epoch_begin, doc_string=False)

#### <a id=on_epoch_begin></a>`on_epoch_begin`
> `on_epoch_begin`(`kwargs`:`Any`)
<a href="https://github.com/fastai/fastai/blob/master/fastai/train.py#L59">[source]</a>

Set back the batchnorm layers on `eval` mode after the model has been set to `train`.

## Undocumented Methods - Methods moved below this line will intentionally be hidden

In [ ]:
show_doc(one_cycle_scheduler)